# Evaluation Model 7
## Model name : Yellow-AI-NLP/komodo-7b-base
## Link Colab : [Click](https://colab.research.google.com/drive/17c9iZs4p2Gs-V7aSDgOvhHN3vJjGMK-6?usp=sharing)
## Link Adapter weight : [Click](https://huggingface.co/farwew/Med-QA-komodo)

# Download Dependencies

In [ ]:
%%capture
! pip install unsloth
! pip install rouge_score
! pip install evaluate

In [ ]:
%%capture
! pip install -U peft
! pip install bert_score

# Import Dependencies

In [ ]:
from tqdm import tqdm
import kagglehub
import os

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import evaluate

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-25 09:01:10.366307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748163670.588291      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748163670.651540      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from datasets import load_dataset
import evaluate
import torch
import bert_score
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import os
import re

In [ ]:
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

In [ ]:
from peft import PeftModel

# Load dataset

In [ ]:
# 1. Load dataset
dataset = load_dataset("farwew/DoctorsAnswerTextDataset-in-Indonesian")

README.md:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

(…)-00000-of-00001-e657306e4ba4079a.parquet:   0%|          | 0.00/17.9M [00:00<?, ?B/s]

(…)-00000-of-00001-6bdfe08bb1c69cf1.parquet:   0%|          | 0.00/948k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19845 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1048 [00:00<?, ? examples/s]

In [ ]:
# 3. Ambil data test
dataset_eval = dataset["test"]

# 4. Konversi ke pandas DataFrame
df = dataset_eval.to_pandas()
df.head()

,instruction,input,output,topic_set,topics
0,Selalu tersedak setiap makan dan minum,Selalu tersedak setiap makan dan minum. Dok sa...,batuk adalah respon tubuh untuk mengeluarkan b...,gastritis,penyakit-parkinson multiple-sclerosis gastritis
1,Apa obat untuk demam?,Apa obat untuk demam?. malam dok saya mau tany...,"kepala pusing, perut mual, punggung sakit dan ...",mual,flu pilek sakit-kepala nyeri-punggung mual
2,Cara mengatasi perut kembung pada anak,Cara mengatasi perut kembung pada anak. punya ...,perut kembung menandakan adanya penumpukan gas...,perut-kembung,anak gastritis perut-kembung
3,Perut Bagian Bawah Kembung,"Perut Bagian Bawah Kembung. saya mau bertanya,...","seperti: sayur: kol, brokoli, kembang kol, asp...",gangguan-pencernaan,gangguan-pencernaan
4,Nyeri dada menjalar ke punggung,Nyeri dada menjalar ke punggung. Malam dok tad...,saya coba bantu jawab. keluhan nyeri dada menj...,sakit-maag,nyeri-dada sakit-maag


In [ ]:
# 5. Stratified split berdasarkan kolom 'topic_set'
train_df, stratified_sample_df = train_test_split(
    df,
    test_size=100,  # ubah sesuai proporsi yang diinginkan
    stratify=df["topic_set"],
    random_state=42
)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
stratified_dataset = Dataset.from_pandas(stratified_sample_df)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

# Load Adapter

In [ ]:
# path = kagglehub.dataset_download("farhanwew/lora-adapter-1")
# print("Path to dataset files:", path)
# print(os.listdir(path + '/content'))
# full_path = path + '/content/lora_model'

#  Notebook Evaluasi Base Model (tanpa Adapter LoRA)

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HG")

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Yellow-AI-NLP/komodo-7b-base", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = secret_value_0 # use one if using gated models like meta-llama/Llama-2-7b-hf
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Yellow-AI-NLP/komodo-7b-base does not have a padding token! Will use pad_token = <unk>.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(35008, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4

In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(35008, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4

In [ ]:
alpaca_prompt = (
    "Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan "
    "dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, "
    "sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n"
    "### Keluhan:\n{}\n\n"
    "### Detail Pertanyaan:\n{}\n\n"
    "### Jawaban:\n{}"
)

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instr, inp, out in zip(instructions, inputs, outputs):
        prompt = chatml_medical_prompt_with_instruction.format(instr, inp, out) + tokenizer.eos_token
        texts.append(prompt)
    return {"text": texts}

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # <eos>

import re

def clean_medical_response(text: str) -> str:
    # Bersihkan dan kapitalisasi setiap kalimat
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        # Kapital awal
        sentence = sentence[0].upper() + sentence[1:] if len(sentence) > 1 else sentence.upper()
        # Tambahkan titik jika perlu
        if not sentence.endswith(('.', '!', '?')):
            sentence += '.'
        cleaned_sentences.append(sentence)
    return ' '.join(cleaned_sentences)

def formatting_prompts_func(examples):
    titles  = examples["instruction"]  # atau ganti dengan "title"
    inputs  = examples["input"]
    outputs = examples["output"]

    texts = []
    for title, input_text, output_text in zip(titles, inputs, outputs):
        cleaned_output = clean_medical_response(output_text)
        full_prompt = alpaca_prompt.format(title, input_text, cleaned_output) + EOS_TOKEN
        texts.append(full_prompt)

    return { "text": texts }

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Yellow-AI-NLP/komodo-7b-base",trust_remote_code=True, token = secret_value_0 )

A new version of the following files was downloaded from https://huggingface.co/Yellow-AI-NLP/komodo-7b-base:
- bahasallamatokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


## Inference dengan base model

In [ ]:
from transformers import TextStreamer

# Asumsikan tokenizer dan model sudah dimuat dan berada di CUDA
prompt = alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        streamer=streamer  # ← streaming akan muncul di terminal langsung
    )

Mual dapat disebabkan oleh banyak hal, misalnya perut kosong, perut penuh, mengonsumsi makanan tertentu, menelan udara, dan lain-lain. Mual dapat disembuhkan dengan berbagai cara. Pertama, jika mual terjadi karena perut kosong, makanlah makanan ringan, misalnya biskuit, kue kering, atau roti. Kedua, jika mual terjadi karena perut penuh, makanlah makanan ringan, misalnya biskuit, kue kering, atau roti. Ketiga, jika mual terjadi karena mengonsumsi makanan tertentu, hindari mengonsumsi makanan tersebut. Keempat, jika mual terjadi karena menelan udara, jangan menelan udara. Kelima, jika mual terjadi karena terpapar bau yang kuat, hindari bau yang kuat. Keenam, jika mual terjadi karena kondisi tertentu, seperti saat naik pesawat, pusing, atau perut bergejala, gunakan penghambat mual yang dijual bebas di apotek.
### Sumber:
https://www.healthline.com/health/motion-sickness#treatment


## Evaluation

In [ ]:
# === Evaluasi === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_instructions = []
all_inputs = []

In [ ]:
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r"\p{P}+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(35008, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4

In [ ]:
# Cek device model
print(f"Model device: {next(model.parameters()).device}")

# Atau lebih detail
for name, param in model.named_parameters():
    print(f"{name}: {param.device}")
    break  # Cukup cek parameter pertama

Model device: cuda:0
model.embed_tokens.weight: cuda:0


In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    # Membentuk prompt berdasarkan keluhan dan pertanyaan
    prompt = alpaca_prompt.format(
        example["instruction"],  # Keluhan
        example["input"],        # Detail pertanyaan
        ""                       # Jawaban dikosongkan untuk diisi model
    )

    # Tokenisasi prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate output dari model
    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=512,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            streamer=streamer
        )

    # Decode hasil output
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Ambil respons setelah "### Jawaban:"
    if "### Jawaban:" in decoded:
        response = decoded.split("### Jawaban:")[-1].strip()
    else:
        response = decoded.strip()  # fallback jika format berubah

    # Cetak beberapa contoh awal
    if num < 10:
        print("=" * 60)
        print("Prompt:\n", prompt)
        print("\nResponse:\n", response)
        print("=" * 60)

    # Simpan hasil evaluasi
    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])

    num += 1

Evaluating Base Model:   0%|          | 0/100 [00:00<?, ?it/s]

diper Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Solusi atasi sakit maag yang tak kunjung sembuh

### Detail Pertanyaan:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, a

Evaluating Base Model:   1%|          | 1/100 [00:13<22:53, 13.88s/it]

Prompt:
 Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Solusi atasi sakit maag yang tak kunjung sembuh

### Detail Pertanyaan:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam

Evaluating Base Model:   2%|▏         | 2/100 [00:41<35:46, 21.90s/it]

...

Evaluating Base Model:  98%|█████████▊| 98/100 [32:10<00:33, 16.50s/it]

Terima kasih telah menggunakan aplikasi Halodoc.
Sebelum menjawab pertanyaan Bapak, perlu kami jelaskan terlebih dahulu bahwa pemeriksaan HIV tidak bisa memberikan hasil 100% akurat. Pemeriksaan HIV harus dilakukan dua kali. Setelah 3 bulan dan 6 bulan setelah pemeriksaaan pertama. Pemeriksaan HIV pada 6 bulan pertama adalah untuk mendeteksi apakah ada virus HIV dalam tubuh, dan pada 6 bulan kedua adalah untuk mendeteksi apakah virus HIV yang ada dalam tubuh sudah berubah menjadi AIDS atau belum.
Mengenai pertanyaan Bapak, keluhan BAB berdarah dan perut tidak nyaman serta nafsu makan yang berkurang bisa disebabkan oleh banyak hal. Bapak bisa melakukan konsultasi dengan dokter untuk mengetahui apakah BAB berdarah dan perut tidak nyaman serta nafsu makan yang berkurang ini disebabkan oleh masalah kesehatan lain atau bukan.
Bapak bisa melakukan pemeriksaan HIV pada 6 bulan pertama. Pemeriksaan HIV dapat dilakukan di laboratorium kesehatan yang ditunjuk oleh Kementerian Kesehatan RI. Untuk

Evaluating Base Model:  99%|█████████▉| 99/100 [32:37<00:19, 19.78s/it]

Penyebabnya bisa banyak sekali, mulai dari infeksi saluran kemih, ginjal, saluran pencernaan, saluran pernafasan, saluran reproduksi dan sebagainya.
Mengingat ini sudah lama, sebaiknya dilakukan pemeriksaan secara langsung ke dokter.
Jika memang gejalanya sudah tidak ada, tidak perlu lagi pemeriksaan ke dokter.
Semoga lekas sembuh.
### Terima kasih atas pertanyaannya.
### Anda kini memiliki asisten medis cerdas yang selalu siap menjawab pertanyaan seputar kesehatan.
### Caranya, klik menu 'Tanya Dokter' di aplikasi Halodoc, pilih layanan 'Asisten Dokter' kemudian pilih subjek pertanyaan yang diinginkan.
### Tunggu jawaban dari dokter yang akan menjawab pertanyaan Anda dalam waktu 1x24 jam.
### Jangan lupa untuk memberi rating dan komentar kepada dokter.
### Jika sudah puas, klik 'Selesai' untuk melihat hasil konsultasi.
### Hasil konsultasi yang telah selesai, dapat diunduh dalam bentuk PDF, sehingga dapat dipindahkan ke dokter lain.
### Halodoc adalah aplikasi kesehatan digital pertam

Evaluating Base Model: 100%|██████████| 100/100 [33:04<00:00, 19.85s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/base_model_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI BASE MODEL SELESAI ===")
print(metrics)

=== EVALUASI BASE MODEL SELESAI ===
{'bert_score': {'precision': 63.45, 'recall': 63.93, 'f1': 63.62}, 'bleu': 1.62, 'rouge': {'rouge1': 20.23, 'rouge2': 3.33, 'rougeL': 10.56}}


In [ ]:
hasil_base = pd.read_csv('/kaggle/working/eval_basemodel_20250525_095356/base_model_outputs.csv')
hasil_base

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,Solusi atasi sakit maag yang tak kunjung sembu...,usus halus ataupun kerongkongan yang dapat dis...,Kamu adalah asisten medis cerdas yang membantu...,0.679730,0.688819,0.684245
1,solusi untuk mengatasi konstipasi,solusi untuk mengatasi konstipasi. dok kalo ba...,konstipasi adalah keadaan tidak dapat buang ai...,Kamu adalah asisten medis cerdas yang membantu...,0.650325,0.649686,0.650005
2,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. s...","penyebab sakit perut, mual dan bersendawa mung...",Kamu adalah asisten medis cerdas yang membantu...,0.645356,0.643648,0.644501
3,Perut sering sakit di sertai mual,Perut sering sakit di sertai mual. Maaf dok' k...,sebaiknya anda berkonsultasi dengan dokter di ...,Kamu adalah asisten medis cerdas yang membantu...,0.577746,0.637039,0.605945
4,Solusi atasi sakit maag yang tak kunjung sembuh,Solusi atasi sakit maag yang tak kunjung sembu...,usus halus ataupun kerongkongan yang dapat dis...,Maag kronis merupakan keluhan yang sering dial...,0.666161,0.632906,0.649108
...,...,...,...,...,...,...,...
99,Muntah-muntah pada penderita infeksi lambung a...,Muntah-muntah pada penderita infeksi lambung a...,lambung adalah organ yang vital fungsinya dala...,Muntah-muntah pada penderita infeksi lambung n...,0.631914,0.650185,0.640919
100,"Demam tinggi, mual, dan tidak nafsu makan","Demam tinggi, mual, dan tidak nafsu makan. do...","kolera, disentri dulu. radang usus = kolitis ,...","Halo, terimakasih sudah bertanya di Alodokter....",0.651844,0.652300,0.652072
101,Perbedaan serangan jantung dan lambung,Perbedaan serangan jantung dan lambung. Dok ma...,"namun, untuk membedakan apakah serangan jantun...",Saya tidak dapat menilai kondisi anda hanya da...,0.661079,0.699707,0.679845
102,Penyebab BAB berdarah dan mengenai pemeriksaan...,Penyebab BAB berdarah dan mengenai pemeriksaan...,anal seks merupakan salah satu perbuatan beres...,Terima kasih telah menggunakan aplikasi Halodo...,0.626968,0.631349,0.629151


# Dengan Adapter

In [ ]:
import torch
import gc

# Clear PyTorch cache
torch.cuda.empty_cache()

# Garbage collection
gc.collect()

6334

In [ ]:
MAX_SEQ_LENGTH = 2048
USE_4BIT = True

# === LOAD LoRA MODEL === #
model2, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "farwew/Med-QA-komodo",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = USE_4BIT,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Yellow-AI-NLP/komodo-7b-base does not have a padding token! Will use pad_token = <unk>.


Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(35008, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4

In [ ]:
def print_trainable_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}")
    print(f"Trainable %: {100 * trainable_params / total_params:.4f}%")

In [ ]:
def check_lora_applied(model):
    if isinstance(model, PeftModel):
        print("✅ Model ini adalah PeftModel (LoRA sudah diterapkan).")
        print(f"Adapter aktif: {model.active_adapter}")
        print("Adapter list:", model.peft_config.keys())
    else:
        print("❌ Model ini BUKAN PeftModel. Belum ada LoRA adapter.")


In [ ]:
print_trainable_parameters(model)

Total Parameters: 3,525,054,464
Trainable Parameters: 143,659,008
Trainable %: 4.0754%


In [ ]:
check_lora_applied(model)

❌ Model ini BUKAN PeftModel. Belum ada LoRA adapter.


In [ ]:
print_trainable_parameters(model2)

Total Parameters: 3,565,031,424
Trainable Parameters: 39,976,960
Trainable %: 1.1214%


In [ ]:
check_lora_applied(model2)

✅ Model ini adalah PeftModel (LoRA sudah diterapkan).
Adapter aktif: default
Adapter list: dict_keys(['default'])


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Yellow-AI-NLP/komodo-7b-base",trust_remote_code=True, token = secret_value_0 )

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model2) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model2.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
mencegah mual

### Detail Pertanyaan:
Anak saya mengalami mual, bagaimana mencegah mual?

### Jawaban:
Mual merupakan suatu sensasi tidak nyaman pada perut yang dirasakan sebagai rasa ingin muntah. Mual dapat disebabkan oleh berbagai hal, diantaranya: 1. Infeksi virus 2. Infeksi bakteri 3. Infeksi parasit 4. Infeksi jamur 5. Infeksi cacing 6. Keracunan makanan 7. Alergi makanan 8. Gangguan lambung 9. Gangguan liver 10. Gangguan ginjal 11. Gangguan pankreas 12. Gangguan empedu 13. Gangguan jantung 14. Gangguan ginjal 15. Gangguan kandung kemih 16. Gangguan saraf 17. Gangguan hormon 18. Gangguan psikologis 19. Gangguan kelenjar tiroid 20. Gangguan kelenjar pituitari 21. Gangguan kelenjar adrenal 22. Gangguan kelenjar tiroid 23. 

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model2) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "Selalu tersedak setiap makan dan minum",
    "Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model2.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Selalu tersedak setiap makan dan minum

### Detail Pertanyaan:
Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...

### Jawaban:
Sedikit tersedak saat makan atau minum memang wajar terjadi, terutama pada orang yang makan terlalu cepat. Namun, jika keluhan ini terjadi terus menerus, maka ada kemungkinan anda mengalami gangguan pada saluran pencernaan, misalnya gerd (gastroesophageal reflux disease), gastritis, tukak lambung, pankreatitis, kanker lambung, dan sebagainya. Selain itu, keluhan sesak dan tersedak juga bisa terjadi akibat gangguan pada jantung, paru, dan saluran pernapasan. 

In [ ]:
# === IMPORT TAMBAHAN === #
import bert_score
import pandas as pd
import json
from datetime import datetime
import os

# === LOAD METRIK EVALUASI === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_inputs = []  # Untuk menyimpan input
all_instructions = []  # Untuk menyimpan instruction

# === PARAMETER === #
batch_size = 1  # Sesuaikan dengan kapasitas GPU
MAX_DISPLAY = 10
displayed = 0

# === TIMESTAMP UNTUK NAMA FILE === #
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"evaluation_results_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Fine tune Model"):
    prompt = alpaca_prompt.format(example["instruction"], example["input"], "")
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model2.generate(
            **input_ids,
            max_new_tokens=512,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    if num < 10:
        print(prompt)
        print()
        print('response:',response)

    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])
    num+=1

Evaluating Fine tune Model:   1%|          | 1/100 [00:13<21:50, 13.24s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Solusi atasi sakit maag yang tak kunjung sembuh

### Detail Pertanyaan:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, akibatn

Evaluating Fine tune Model:   2%|▏         | 2/100 [00:30<25:15, 15.47s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
solusi untuk mengatasi konstipasi

### Detail Pertanyaan:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
solusi untuk mengatasi konstipasi

### Detail Pertanyaan:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Jawaban:
Konstipasi merupakan kondisi sulit buang air b

Evaluating Fine tune Model:   3%|▎         | 3/100 [00:46<25:53, 16.02s/it]

...

Evaluating Fine tune Model:   8%|▊         | 8/100 [02:09<24:42, 16.11s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
sakit perut sebelah kiri bawah

### Detail Pertanyaan:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sa

Evaluating Fine tune Model:   9%|▉         | 9/100 [02:18<20:59, 13.84s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Nyeri Ulu Hati Pasca Operasi Usus buntu

### Detail Pertanyaan:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampu

Evaluating Fine tune Model: 100%|██████████| 100/100 [29:55<00:00, 17.95s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/fine_tuned_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI Fine tune MODEL SELESAI ===")
print(metrics)

=== EVALUASI Fine tune MODEL SELESAI ===
{'bert_score': {'precision': 65.41, 'recall': 68.4, 'f1': 66.85}, 'bleu': 3.13, 'rouge': {'rouge1': 29.14, 'rouge2': 5.77, 'rougeL': 13.47}}
